<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Mitigating Indirect Bias with Watson OpenScale

This notebook demonstrates the advances concepts of fairness detection using Indirect Bias mechanism using IBM Watson OpenScale.

We make use of the adult income dataset, where the attributes `age`, `sex` and `race` are not used to train the machine learning model that we are building as part of this notebook. But we will configure fairness against these attributes (specifically `age` and `sex`) to check if the model is indirectly behaving in a biased manner with these protected (sensitive) attributes.

## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale and an instance of IBM Watson Machine Learning using the Cloud catalog.

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, click your name, scroll down to the API Keys section, and click Create an IBM Cloud API key. Give your key a name and click Create, then copy the created key and paste it below.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [Instructions](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

<b>How to get api key using console:</b>

<li> bx login --sso
<li> bx iam api-key-create 'my_key'

## Credentials for IBM Cloud services

### Retrieve your IBM Cloud API key

1.	From the IBM Cloud toolbar, click your Account name, such as <Your user name>’s Account.
1.	From the Manage menu, click Access (IAM).
1.	In the navigation bar, click IBM Cloud API keys.
1.	Click the Create an IBM Cloud API key button.
1.	Type a name and description and then click Save.
1.	Copy the newly created API key and paste it into your notebook in the following **CLOUD_API_KEY** code box, which is the first code box.

    Note: replace everything between the two sets of double quotation marks (").

In [ ]:
CLOUD_API_KEY = ""

### Retrieve your Watson Machine Learning credentials

1.	Go to the IBM Cloud dashboard.
1.	In the Resource summary section, click Services.
1.	Click Machine Learning.
1.	In the navigation pane, click Service credentials.
1.	Click the New credential button.
1.	Copy your credentials by clicking the copy icon.
1.	Return to the notebook editor and update the credentials by replacing the sample credentials with your own in the below cell

   **Note**: You need to replace everything including the opening bracket ({) and the closing bracket (}).
   
   **IMPORTANT**: If you are reusing a WML instance that is already bound to Watson OpenScale. Please specify that instance credentials in `wml_credentials`

In [ ]:

wml_credentials = {
  
}

In [ ]:
DB_CREDENTIALS = None
KEEP_MY_INTERNAL_POSTGRES = True
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

# Package installation

The following opensource packages must be installed into this notebook instance so that they are available to use during processing.

In [ ]:
!rm -rf $PIP_BUILD
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install psycopg2-binary | tail -n 1
!pip install pyspark==2.3.0

Restart the kernel to assure the new libraries are being used.

# Load and explore data

## Explore data

In [ ]:
model_name = 'Implicit Bias v2'
deployment_name = 'Implicit Bias Deploy v2'

In [ ]:
import pandas as pd
from pyspark import SparkContext, SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.sql.types import StructType, DoubleType, StringType, ArrayType

In [ ]:
#import data

In [ ]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_57b5eb6e7b5d4b90a8e83736129c9010 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='***',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='***')

body = client_57b5eb6e7b5d4b90a8e83736129c9010.get_object(Bucket='resources-donotdelete-pr-lbyypdyr2le8tz',Key='final_8_noincome.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

pd_data = pd.read_csv(body, sep=",", header=0)
pd_data.head()


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.getOrCreate()
spark_df=spark.createDataFrame(pd_data)
#spark_df = spark.read.csv(path="../final_8.csv", sep=",", header=True, inferSchema=True)
spark_df.head()

In [ ]:
si_BusinessTravel = StringIndexer(inputCol='BusinessTravel', outputCol='BusinessTravel_IX')
si_Department = StringIndexer(inputCol='Department', outputCol='Department_IX')
si_Education = StringIndexer(inputCol='Education', outputCol='Education_IX')
si_EducationField = StringIndexer(inputCol='EducationField', outputCol='EducationField_IX')
si_RelevantEducationLevel = StringIndexer(inputCol='RelevantEducationLevel', outputCol='RelevantEducationLevel_IX')
si_JobRole = StringIndexer(inputCol='JobRole', outputCol='JobRole_IX')
si_JobLevel = StringIndexer(inputCol='JobLevel', outputCol='JobLevel_IX')
si_MaritalStatus = StringIndexer(inputCol='MaritalStatus', outputCol='MaritalStatus_IX')
si_OverTime = StringIndexer(inputCol='OverTime', outputCol='OverTime_IX')
si_RequestedBenefits = StringIndexer(inputCol='RequestedBenefits', outputCol='RequestedBenefits_IX')
si_PreferredSkills = StringIndexer(inputCol='PreferredSkills', outputCol='PreferredSkills_IX')
si_JobType = StringIndexer(inputCol='JobType', outputCol='JobType_IX')
si_SalaryExpectation = StringIndexer(inputCol='SalaryExpectation', outputCol='SalaryExpectation_IX')

si_InterviewScore = StringIndexer(inputCol='InterviewScore', outputCol='InterviewScore_IX')
si_ResumeScore = StringIndexer(inputCol='ResumeScore', outputCol='ResumeScore_IX')

#si_MonthlyIncome = StringIndexer(inputCol='MonthlyIncome', outputCol='MonthlyIncome_IX')

si_Gender = StringIndexer(inputCol='Gender', outputCol='Gender_IX')
si_Ethnicity = StringIndexer(inputCol='Ethnicity', outputCol='Ethnicity_IX')

si_Label = StringIndexer(inputCol="HIRED", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [ ]:
#payload sample
(train_data_payload, test_data_payload) = spark_df.randomSplit([0.9, 0.1], 24)

In [ ]:
columns_to_drop = ['Gender', 'Ethnicity']
spark_df_tmp = spark_df.drop(*columns_to_drop)

In [ ]:
(train_data, test_data) = spark_df_tmp.randomSplit([0.9, 0.1], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

In [ ]:
va_features = VectorAssembler(
inputCols=["Age", "BusinessTravel_IX", "Department_IX", "DistanceFromHome",
           "Education_IX", "EducationField_IX", "RelevantEducationLevel_IX", "JobLevel_IX", "JobRole_IX"
           , "MaritalStatus_IX","NumCompaniesWorked", "OverTime_IX",
           "InterviewScore_IX", "ResumeScore_IX", "RequestedBenefits_IX", "TotalWorkingYears", "PreferredSkills_IX",
           "YearsAtCurrentCompany","RelevantExperience","JobType_IX","SalaryExpectation_IX"], outputCol="features")

In [ ]:
classifier=GBTClassifier(featuresCol="features")#classifier = RandomForestClassifier(featuresCol="features")

In [ ]:
pipeline = Pipeline(stages=[si_BusinessTravel, si_Department,si_Education, si_EducationField,si_RelevantEducationLevel, si_JobRole,si_JobLevel, si_MaritalStatus,
        si_OverTime,si_InterviewScore,si_ResumeScore,si_RequestedBenefits,si_PreferredSkills,si_JobType,si_SalaryExpectation, si_Label, va_features, classifier, label_converter])

In [ ]:
model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)

# Save and deploy the model

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(model_name),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.8
        }
    ]
}

In [ ]:
model = pipeline.fit(spark_df_tmp)

In [ ]:
model_uid = None
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_uid = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == deployment_name:
        print('Deleting existing deployment with id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting existing model with id', model_uid)
        wml_client.repository.delete(model_uid)

print("Storing model ...")

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=spark_df_tmp, pipeline=pipeline)
model_uid = wml_client.repository.get_model_uid(published_model_details)
print("Done")

In [ ]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if deployment_name == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=deployment_name, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

In [ ]:
# paylaod import

payload_data = pd.read_csv(body)
payload_data.head()

In [ ]:
protected_attributes=['Ethnicity','Gender']

In [ ]:
cols_to_remove = ['HIRED']
cols_to_remove.extend(protected_attributes)
cols_to_remove

## Create the meta data frame capturing the sensitive data

In [ ]:
meta_df = payload_data[protected_attributes].copy()
meta_fields = meta_df.columns.tolist()
meta_values = meta_df[meta_fields].values.tolist()

## Construct the scoring payload comprising the meta fields

In [ ]:
def get_scoring_payload(no_of_records_to_score = 1):
    meta_payload = {
        "fields": meta_fields,
        "values": meta_values[:no_of_records_to_score]
    }

    for col in cols_to_remove:
        if col in payload_data.columns:
            del payload_data[col] 

    fields = payload_data.columns.tolist()
    values = payload_data[fields].values.tolist()

    payload_scoring = {"fields": fields,"values": values}

    payload_scoring = {
        "fields": fields,
        "values": values[:no_of_records_to_score],
        "meta": meta_payload
    }

    #import json
    #print(json.dumps(payload_scoring, indent=None))    
    return payload_scoring

## Method to perform scoring

In [ ]:
def payload_logging(no_of_records_to_score = 1):
    payload_scoring = get_scoring_payload(no_of_records_to_score)
    scoring_endpoint = None

    for deployment in wml_client.deployments.get_details()['resources']:
        if deployment_uid in deployment['metadata']['guid']:
            scoring_endpoint = deployment['entity']['scoring_url']

    print(scoring_endpoint)    
    scoring_response = wml_client.deployments.score(scoring_endpoint, payload_scoring)
    
    print('Single record scoring result:', '\n fields:', scoring_response['fields'], '\n values: ', scoring_response['values'][0])
    #print(json.dumps(scoring_response, indent=None))

## Score the model and print the scoring response

In [ ]:
payload_logging(no_of_records_to_score = 1)

# Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

### Get Watson OpenScale GUID

Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.
1. Please update the `url` in the below WOS_CREDENTIALS payload as per the environment that you are using.
2. Please update the `DASHBOARD_URL` in the below cell as per the environment that you are using.

In [ ]:
from ibm_ai_openscale.utils import get_instance_guid


WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.test.cloud.ibm.com"
}
DASHBOARD_URL = "https://aiopenscale.test.cloud.ibm.com"

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

In [ ]:
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
ai_client.version

## Generate an IAM token
### The following is a function that will generate an IAM access token used to interact with the Watson OpenScale APIs

In [ ]:
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth("bx", "bx")
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": CLOUD_API_KEY
    }
    response = requests.post(IAM_URL, data=data, headers=headers, auth=auth)
    json_data = response.json()
    iam_access_token = json_data['access_token']
    return iam_access_token

## Create schema and datamart

### Set up datamart
Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were not supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there unless there is an existing datamart and the KEEP_MY_INTERNAL_POSTGRES variable is set to True. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the Income classifier model will be removed from OpenScale monitoring.

In [ ]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        try:
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
        except:
            print('Setup failed, trying Db2 setup')
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS, schema=DB_CREDENTIALS['username'])

In [ ]:
data_mart_details = ai_client.data_mart.get_details()

In [ ]:
data_mart_details

## Bind WML machine learning instance as Pre-Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "WML Pre-Prod" already exists, this code will delete that binding a create a new one.

In [ ]:
all_bindings = ai_client.data_mart.bindings.get_details()['service_bindings']
existing_binding = False
for binding in all_bindings:
    binding_uid = binding['metadata']['guid']
    if binding['metadata']['guid'] == wml_credentials['instance_id']:
        existing_binding = True
        break

if not existing_binding:
    binding_uid = ai_client.data_mart.bindings.add('WML Prod', WatsonMachineLearningInstance(wml_credentials))
    
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

In [ ]:
print(binding_uid)
ai_client.data_mart.bindings.get_details(binding_uid)

In [ ]:
ai_client.data_mart.bindings.list_assets()

## Patch binding as production

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

payload = [
 {
   "op": "replace",
   "path": "/operational_space_id",
   "value": "production"
 }
]

In [ ]:
SERVICE_PROVIDER_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/service_providers/{1}".format(WOS_GUID, binding_uid)
response = requests.patch(SERVICE_PROVIDER_URL, json=payload, headers=headers)
json_data = response.json()
print(json_data)

## Remove existing prod subscription

### This code removes previous subscription that matches the name German Credit Risk Model - Prod as it is expected this subscription is created only via this notebook.

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == model_name:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', sub_name)

In [ ]:
feature_columns=list(pd_data.drop(['HIRED','Gender','Ethnicity'],axis=1))
feature_columns

In [ ]:
categorical_features = pd_data[feature_columns].select_dtypes(include=['object']).columns.tolist()
categorical_features

In [ ]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    binding_uid=binding_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='HIRED',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = feature_columns,
    categorical_columns = categorical_features
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == model_name:
            subscription = ai_client.data_mart.subscriptions.get(sub)

In [ ]:
ai_client.data_mart.subscriptions.list()

In [ ]:
subscription.uid

## Patch the training data reference to the challenger subscription

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

# training_data_reference = {
#   "connection": {
#     "database_name": "***",
#     "hostname": "***",
#     "port": "50001",
#     "password": "***",
#     "username": "***",
#     "ssl": True
#   },
#   "location": {
#     "schema_name": "NMX87075",
#     "table_name": "ADULT_INCOME"
#   },
#   "type": "db2"
# }

training_data_reference = {
                "connection": {
                    "api_key": "***",
                    "iam_url": "https://iam.cloud.ibm.com/oidc/token",
                    "resource_instance_id": "***",
                    "url": "https://s3.us.cloud-object-storage.appdomain.cloud"
                },
                "location": {
                    "bucket": "***",
                    "file_format": "csv",
                    "file_name": "final_8_noincome.csv",
                    "firstlineheader": True,
                    "infer_schema": "1"
                },
                "type": "cos"
            }

payload = [
 {
   "op": "replace",
   "path": "/asset_properties/training_data_reference",
   "value": training_data_reference
 }
]

In [ ]:
SUBSCRIPTION_URL = WOS_CREDENTIALS["url"] + "/v1/data_marts/{0}/service_bindings/{1}/subscriptions/{2}".format(WOS_GUID, binding_uid, subscription.uid)

response = requests.patch(SUBSCRIPTION_URL, json=payload, headers=headers)
json_data = response.json()
print(json_data)

## Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [ ]:
payload_logging(no_of_records_to_score = 100)

In [ ]:
time.sleep(2)
subscription.payload_logging.get_records_count()

## Fairness, drift monitoring and explanations
###  Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, sex and age. In each case, we must specify:

Which model feature to monitor One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%) Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

### Create Fairness Monitor Instance

In [ ]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

MONITOR_INSTANCES_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances".format(WOS_GUID)
MONITOR_INSTANCES_URL

In [ ]:
fairness_paylaod = {
    "data_mart_id": WOS_GUID,
    "monitor_definition_id": "fairness",
    "parameters": {
        "features": [
            {
                "feature": "Gender",
                "majority": ["Male"],
                "minority": ["Female"]
            },
            {
                "feature": "Ethnicity",
                "majority": ["non-minority"],
                "minority": ["minority"]
            }
        ],
        "favourable_class": ["YES"],
        "unfavourable_class": ["NO"],
        "min_records": 100
    },
    "target": {
        "target_type": "subscription",
        "target_id": subscription.uid
    },
    "thresholds": [
        {
            "metric_id": "fairness_value",
            "specific_values": [
                
                {
                    "applies_to": [
                        {
                            "type": "tag",
                            "value": "Gender",
                            "key": "feature"
                        }
                    ],
                    "value": 80
                },
                {
                    "applies_to": [
                        {
                            "type": "tag",
                            "value": "Ethnicity",
                            "key": "feature"
                        }
                    ],
                    "value": 80
                }
            ],
            "type": "lower_limit",
            "value": 80
        }
    ]
}
#print(json.dumps(fairness_paylaod, indent=2))

In [ ]:
response = requests.post(MONITOR_INSTANCES_URL, json=fairness_paylaod, headers=headers, verify=False)
json_data = response.json()
print()
print(json_data)
print()

### Get Fairness Monitor Instance

In [ ]:
MONITOR_INSTANCES_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances?target.target_id={1}&target.target_type=subscription".format(WOS_GUID, subscription.uid)
print(MONITOR_INSTANCES_URL)

response = requests.get(MONITOR_INSTANCES_URL, headers=headers)
monitor_instances = response.json()["monitor_instances"]

fairness_monitor_instance_id = None

if monitor_instances is not None:
    for monitor_instance in monitor_instances:
        if "entity" in monitor_instance and "monitor_definition_id" in monitor_instance["entity"]:
            monitor_name = monitor_instance["entity"]["monitor_definition_id"]
            if "metadata" in monitor_instance and "id" in monitor_instance["metadata"]:
                id = monitor_instance["metadata"]["id"]
                if monitor_name == "fairness":
                    fairness_monitor_instance_id = id
                    
print("Fairness monitor instance id - {0}".format(fairness_monitor_instance_id))

### Get existing fairness monitoring run id

When we configure fairness monitor, the underlying OpenScale fairness service will submit the first evaluation. For this evaluation in the below cell, we are trying to get the monitoring run it.

In [ ]:
# sleep for few seconds for the fairneess monitoring check to be submitted.
time.sleep(2)

def get_monitoring_run_id(monitor_instance_id):
    monitoring_run_id = None
    MONITOR_INSTANCE_RUNS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs".format(WOS_GUID, monitor_instance_id)
    response = requests.get(MONITOR_INSTANCE_RUNS_URL, headers=headers, verify=False)
    monitoring_runs = response.json()
    
    # there can be only one run
    if monitoring_runs is not None and "runs" in monitoring_runs:
        for run in monitoring_runs["runs"]:
            if "metadata" in run and "id" in run["metadata"]:
                monitoring_run_id = run["metadata"]["id"]
                break
    return monitoring_run_id
        
fairness_monitoring_run_id = get_monitoring_run_id(fairness_monitor_instance_id)
print("Fairness monitoring run id - {0}".format(fairness_monitoring_run_id))

### Function to get the monitoring run details

In [ ]:
def get_monitoring_run_details(monitor_instance_id, monitoring_run_id):
    
    MONITORING_RUNS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs/{2}".format(WOS_GUID, monitor_instance_id, monitoring_run_id)
    response = requests.get(MONITORING_RUNS_URL, headers=headers, verify=False)
    return response.json()

In [ ]:
from datetime import datetime

def check_fairness_run_status(fairness_monitor_instance_id, fairness_monitoring_run_id):
    fairness_run_status = None
    while fairness_run_status != 'finished':
        monitoring_run_details = get_monitoring_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id)
        fairness_run_status = monitoring_run_details["entity"]["status"]["state"]
        if fairness_run_status == "error":
            print(monitoring_run_details)
            break
        if fairness_run_status != 'finished':
            print(datetime.utcnow().strftime('%H:%M:%S'), fairness_run_status)
            time.sleep(10)
    print(fairness_run_status)

check_fairness_run_status(fairness_monitor_instance_id, fairness_monitoring_run_id)

# fairness_run_details = get_monitoring_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id)
# print(json.dumps(fairness_run_details, indent = 2))

In [ ]:
FAIRNESS_DASHBOARD_URL = DASHBOARD_URL + "/aiopenscale/insights/{0}/fairness/Gender?features=fairnessv2,indirect_bias,v2transaction".format(deployment_uid)

In [ ]:
from IPython.display import Markdown as md
md("#### Link to IBM Watson OpenScale Fairness Dashboard: {}".format(FAIRNESS_DASHBOARD_URL))

### Run on-demand Fairness
If you would like to peform an on-demand fairness check, then we need to score a fresh set of data with meta-fields, so that they would be used for indirect bias checking. So the below two cells will score and make sure these records are reached to payload logging table.

In [ ]:
payload_logging(no_of_records_to_score = 200)

In [ ]:
time.sleep(2)
subscription.payload_logging.get_records_count()

### Submit the fairness check

In [ ]:
if fairness_monitor_instance_id is not None:
    MONITOR_RUN_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitor_instances/{1}/runs".format(WOS_GUID, fairness_monitor_instance_id)
    payload = {
        "triggered_by": "user"
    }
    print("Triggering Fairness computation with {}".format(MONITOR_RUN_URL))
    response = requests.post(MONITOR_RUN_URL, json=payload, headers=headers, verify=False)
    json_data = response.json()
    print()
    print(json_data)
    print()
    if "metadata" in json_data and "id" in json_data["metadata"]:
        fairness_monitoring_run_id = json_data["metadata"]["id"]
    print("Done triggering Fairness check")

### Check for its status

In [ ]:
check_fairness_run_status(fairness_monitor_instance_id, fairness_monitoring_run_id)

In [ ]:
from IPython.display import Markdown as md
md("#### To view the latest evaluation of the fairness check, please visit IBM Watson OpenScale Fairness Dashboard: {}".format(FAIRNESS_DASHBOARD_URL))

## Additional data to help debugging

In [ ]:
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))
print("OpenScale Datamart id: {}".format(WOS_GUID))
print("OpenScale Subscription id: {}".format(subscription.uid))
print("OpenScale Fairness Monitor Instance id: {}".format(fairness_monitor_instance_id))
print("OpenScale Fairness Monitoring Run id: {}".format(fairness_monitoring_run_id))
